In [2]:
import pandas as pd

df = pd.read_csv('../../src/train.csv')
print(df.head())

                           image_id data_provider  isup_grade gleason_score
0  0005f7aaab2800f6170c399693a96917    karolinska           0           0+0
1  000920ad0b612851f8e01bcc880d9b3d    karolinska           0           0+0
2  0018ae58b01bdadc8e347995b69f99aa       radboud           4           4+4
3  001c62abd11fa4b57bf7a6c603a11bb9    karolinska           4           4+4
4  001d865e65ef5d2579c190a0e0350d8f    karolinska           0           0+0


In [3]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.2, random_state=42, stratify=df['isup_grade'])

In [4]:
train_df = train.reset_index(drop=True)
val_df = val.reset_index(drop=True)


In [5]:

import pandas as pd
from sklearn.model_selection import StratifiedKFold

# 1) Initialize k-fold column
df["k_fold"] = -1

# 2) Only perform k-fold on samples where image_id is in the training set
mask = df["image_id"].isin(train_df["image_id"])
df_train = df[mask].copy()

# stratification labels
y = df_train["isup_grade"].values

# 3) Prepare index mapping back to original df
train_indices = df_train.index

# 4) Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (_, val_idx) in enumerate(skf.split(df_train, y)):
    val_original_idx = train_indices[val_idx]     # map back to original df rows
    df.loc[val_original_idx, "k_fold"] = fold

# 5) Inspect results
print(df["k_fold"].value_counts())
df.head()

k_fold
-1    2123
 1    1699
 0    1699
 3    1698
 4    1698
 2    1698
Name: count, dtype: int64


,image_id,data_provider,isup_grade,gleason_score,k_fold
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,3
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,-1
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,1
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,1


In [6]:
import os

avaiable_image_ids = set(os.listdir('/mnt/d/YC.Liu/UNI2_h_features/PANDA/panda'))

for i in df["image_id"]:
    if i + '.h5' not in avaiable_image_ids:
        print(i)
        
for i in avaiable_image_ids:
    if i[:-3] not in set(df["image_id"]):
        print(i)

In [7]:
df.to_csv('../../src/panda_k_fold.csv', index=False)